# Blockchain-Based Vehicle Info Recording and Enquiry System 

In [1]:
%reload_ext autoreload
%autoreload 2

# Initial imports
import sys
import requests
from dotenv import load_dotenv

from pprint import pprint

# Import general puspose functions from crypto.py module
from crypto import convertDataToJSON, pinJSONtoIPFS, initContract, w3

import os 
import datetime

load_dotenv()

True

### Creating an instance of the contract that we want to interact with

In [2]:
carsContract = initContract() # initContract() is defined in crypto.py module

Contract Address 0x2aB90971CE9EA7FD4A64C591E16144ebaF1D7279


### Functions to (a) register vehicle (b) register logs and (c) register service details 

In [3]:
# Function to register vechile as NFT in blockchain

def registerVehicle():
    # gather user inputs about his vehicle to be registered as NFT
    owner = input("Enter your wallet address >>> ")
    vin = input("Enter VIN >>> ")
    regdate = input("Enter DMV registration date >>> ")
    mileage = input("Enter miles on the vehicle >>> ")
    image = input("Vehicle image URL >>> ")
    
    # encapsulate vehicle info into an object
    car_object = {
            "1_vin": vin,
            "2_registration_date": regdate,
            "3_mileage": mileage,
            "4_image": image
    }
    
    # convert the vehicle info into JSON structure 
    json_data = convertDataToJSON(car_object)
    
    # pin JSON into IPFS. Save the URI in a local var
    token_uri = pinJSONtoIPFS(json_data)
    
    # display the URI - user can click on this link to check if his info is really saved in IPFS
    print(f"\nToken URI: \nhttps://ipfs.io/ipfs/{token_uri}")
    
    # Finally register the vechile as NFT and save the token id in a local var - This token id will be used to interact with other functions
    token_id = carsContract.functions.registerVehicle(owner, vin, token_uri).call() # registerVehicle is defined in the 
                                                                                    # contract cars_smart_contract.sol which is made
                                                                                    # available though respective ABI json file and 
                                                                                    # contract address
    
    # Display the token_id for future interaction with this system
    print()
    print("----------------------------------------------------------------------------------------------------------")
    print("Please note down the token id for your vehicle - this will be needed for future enquiry about your vehicle")
    print("----------------------------------------------------------------------------------------------------------")
    print(f"TOKEN ID >>> {token_id}")
    
    
    print()
    print("Thanks for your input - 1000 reward tokens will hit your wallet in few mins")
    
    # Execute the smart contract transaction and return the receipt
    tx_hash = carsContract.functions.registerVehicle(owner, vin, token_uri).transact(
        {"from": w3.eth.accounts[0]}
    )
    
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    return receipt

In [24]:
# Function to register logs before and after every shift

def registerLog():
    # logs will be registered for a vehicle which is represented by token_id. Getting the token_id from the user.
    token_id = int(input("\nWhat's your vehicle's Token Id >>> "))
    
    # logs will need two times per shift - one at the start of the shift and then again at the end of the shift
    start_or_end = input("Trip start or end? >>> ")
    
    # Get current date and time
    dt = datetime.datetime.now()
    # Format datetime string
    logtime = dt.strftime("%Y-%m-%d %H:%M:%S")

    # Name of the person entering the log
    name = input("Enter your name >>> ")
    mileage = input("Mileage on the vehicle >>> ")
    safety = input("Safety OK? [Y/N] >>> ")
    comments = input("Any other comments/notes: ")
    
    # encapsulate log info into an object
    log_object = {
        "1_token_id": token_id,
        "2_start_or_end": start_or_end,
        "3_logtime": logtime,
        "4_operator_name": name,
        "5_mileage": mileage,
        "6_safety": safety,
        "7_comments": comments
    }
    
    # convert the vehicle info into JSON structure 
    json_data = convertDataToJSON(log_object)
    
    # pin JSON into IPFS. Save the URI in a local var
    log_uri = pinJSONtoIPFS(json_data)
    
    # display the URI - user can click on this link to check if his info is really saved in IPFS
    print(f"\nLogbook URI: \nhttps://ipfs.io/ipfs/{log_uri}")

    # Execute the smart contract transaction with token_id and IPFS URI for the log 
    # reportLog() function is defined in the smart contract cars_smart_contract.sol
    tx_hash = carsContract.functions.registerLog(token_id, log_uri).transact(
        {"from": w3.eth.accounts[0]}
    )
    
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    print()
    print("\nYour inputs are recorded. Your reward tokens will hit your wallet in next few mins")
    
    return receipt, token_id, log_uri

In [25]:
# Function to register vehicle service details
def registerService():
    # Service details will be registered for a vehicle which is represented by token_id. Getting the token_id from the user.
    token_id = int(input("Token ID of the vehicle: "))
    
    # Get current date and time
    dt = datetime.datetime.now()
    # Format datetime string
    serviceDate = dt.strftime("%Y-%m-%d %H:%M:%S")
    
    servieDetails = input("Service Details: ")
    safety = input("Safety OK? [Y/N] >>> ")
    nextServiceDate = input("Next Service Date: ")
    comments = input("Any other comments/notes: ")
    
    # encapsulate service info into an object
    service_object = {
        "1_token_id": token_id,
        "2_serviceDate": serviceDate,
        "3_servieDetails": servieDetails,
        "4_safety": safety,
        "5_nextServiceDate": nextServiceDate,
        "6_comments": comments
    }
    
    # convert the vehicle info into JSON structure 
    json_data = convertDataToJSON(service_object)
    
    # pin JSON into IPFS. Save the URI in a local var
    service_uri = pinJSONtoIPFS(json_data)

    # display the URI - user can click on this link to check if his info is really saved in IPFS
    print(f"\nService URI: \nhttps://ipfs.io/ipfs/{service_uri}")
    
    # Execute the smart contract transaction with token_id and IPFS URI for the log
    # reportService() function is defined in the smart contract cars_smart_contract.sol
    tx_hash = carsContract.functions.registerService(token_id, service_uri).transact(
        {"from": w3.eth.accounts[0]}
    )
    
    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    print()
    print("Thanks. Your inputs are recorded. Your reward tokens will hit your wallet in next few mins")
    
    return receipt, token_id, service_uri

In [26]:
# function to retrieve vehicle (represented by token_id) info already saved in IPFS 
def getVehicleInfo(token_id):
    # vehicleinfo is a mapping between token_id and IFPS URI. This mapping data structure is defined in the smart contract cars_smart_contract.sol
    vehicleURI = carsContract.functions.vehicleinfo(token_id).call()  # this gets IFPS URI for a token_id
    vehicleInfo = requests.get(f"https://cloudflare-ipfs.com/ipfs/{vehicleURI}")
    return vehicleInfo.json()

In [27]:
# function to retrieve loginfo for a vehicle (token_id) 
def getLogInfo(token_id):
    # one vehicle (token_id) will have many logs - each will have it's own IPFS URI - statement below will get count of URIs for a token_id
    logURICount = carsContract.functions.getLogURICount(token_id).call() # Now we know how many logs we have to retrieve 
    tripno = 0
    # iterate through URIs and get the content in respective URI
    for i in range(logURICount):
        if i%2 == 0:
            tripno += 1
            print(f">>>>>>>> TRIP# {tripno} >>>>>>>>\n")
        logURI = carsContract.functions.getLogURI(token_id, i).call() # getLogURI() is defined in cars_smart_contract.sol
        logInfo = requests.get(f"https://cloudflare-ipfs.com/ipfs/{logURI}") # this gets the content from the URI
        pprint(logInfo.json())
        print("\n")

In [28]:
def getServiceInfo(token_id):
    serviceURICount = carsContract.functions.getServiceURICount(token_id).call() # Now we know how many logs we have to retrieve 
    
    # iterate through URIs and get the content in respective URI
    for i in range(serviceURICount):
        print(f"\n>>>>>>>> SERVICE# {i+1} >>>>>>>>")
        serviceURI = carsContract.functions.getServiceURI(token_id, i).call() # getLogURI() is defined in cars_smart_contract.sol
        serviceInfo = requests.get(f"https://cloudflare-ipfs.com/ipfs/{serviceURI}") # this gets the content from the URI
        
        pprint(serviceInfo.json())

In [32]:
# this component is the starting point of user interaction. Based on the user's intention appropriate function will be called.
# each of these functions will call relevant functions defined in the smart contract cars_smart_contract.sol to get the URI and then 
# extract the content from IPFS and present formatted content
print("Following features are supported by this MVP:")
print("1. Register the vechicle as NFT in Blockchain [OWNER]")
print("2. Enter Log-book entry into Blockchain        [OPERATOR]")
print("3. Enter Service Details into Blockchain      [SERVICE SPECIALIST]")
print("4. Enquire/view Vehicle records               [ATTORNEY, OWNER]")

option = input("\nPick one of the above options [1 or 2 or 3 or 4] >>> ")

if option == "1":
    registerVehicle()
elif option == "2":
    registerLog()
elif option == "3":
    registerService()
elif option == "4":
    token_id = int(input("Token id of the vechile: "))
    car = carsContract.functions.cars(token_id).call()
    
    # VEHICLE INFORMATION
    print("\n==================================================================")
    print("                     Vehicle Information: ")
    print("==================================================================")
    print("VIN ", car[0], "has ", car[1], " Log entrie(s) and", car[2], " Service entrie(s).\n")
    
    vehicleReport = getVehicleInfo(token_id)
    pprint(vehicleReport)
    
    # LOGBOOK HISTORY
    print("\n=========================================")
    print("           LOG-BOOK RECORD: ")
    print("=========================================")
    
    getLogInfo(token_id)
    
    
    # SERVICE HISTORY
    print("\n========================================")
    print("           SERVICE RECORD: ")
    print("========================================")
    
    getServiceInfo(token_id)

else:
    print("Please provide valid input")

Following features are supported by this MVP:
1. Register the vechicle as NFT in Blockchain [OWNER]
2. Enter Log-book entry into Blockchain        [OPERATOR]
3. Enter Service Details into Blockchain      [SERVICE SPECIALIST]
4. Enquire/view Vehicle records               [ATTORNEY, OWNER]



Pick one of the above options [1 or 2 or 3 or 4] >>>  4
Token id of the vechile:  1



                     Vehicle Information: 
VIN  1FMZK04185GA30815 has  4  Log entrie(s) and 1  Service entrie(s).

{'1_vin': '1FMZK04185GA30815',
 '2_registration_date': '10/04/2020',
 '3_mileage': '2000',
 '4_image': 'Not now'}

           LOG-BOOK RECORD: 
>>>>>>>> TRIP# 1 >>>>>>>>
{'1_token_id': 1,
 '2_start_or_end': 'start',
 '3_logtime': '2021-10-07 21:40:24',
 '4_operator_name': 'Tapas Roy',
 '5_mileage': '2500',
 '6_safety': 'Y',
 '7_comments': 'None'}


{'1_token_id': 1,
 '2_start_or_end': 'end',
 '3_logtime': '2021-10-07 21:41:55',
 '4_operator_name': 'Tapas Roy',
 '5_mileage': '3000',
 '6_safety': 'ok',
 '7_comments': 'none'}


>>>>>>>> TRIP# 2 >>>>>>>>
{'1_token_id': 1,
 '2_start_or_end': 'start',
 '3_logtime': '2021-10-07 22:20:26',
 '4_operator_name': 'Tapas',
 '5_mileage': '234',
 '6_safety': 'y',
 '7_comments': 'none'}


{'1_token_id': 1,
 '2_start_or_end': 'end',
 '3_logtime': '2021-10-07 22:20:52',
 '4_operator_name': 'Tapas',
 '5_mileage': '345',
 '6_safety': 'y',
 '